In [ ]:
import pandas as pd
import numpy as np
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import matplotlib.pyplot as plt
from time import sleep
import inspect
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime,timedelta
import datetime
import schedule
import time
from reportlab.lib.pagesizes import letter, mm
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer, Image, PageBreak
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from collections import defaultdict  
from typing import List
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import base64
import json

# Get the search term and tracking period from the user
search_term = input("Please enter the name of the product you want to search: ")
tracking_days = input("Please enter the number of days you want to track the product: ")

# To ensure that the user enters a non-string value 
while not tracking_days.isdigit():
    print("Warning: Please enter a valid integer value for the number of days.")
    tracking_days = input("Please enter the number of days you want to track the product: ")
tracking_days = int(tracking_days)



# Start the web driver and go to the Hepsiburada homepage
options = uc.ChromeOptions()
options.add_argument('--blink-settings=imagesEnabled=false') # disable images for loading of page faster
options.add_argument('--disable-notifications')
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
driver = uc.Chrome(options=options)

url = 'https://www.hepsiburada.com/'
driver.get(url)
wait = WebDriverWait(driver, 15)

# Close cookies bar
wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

# Enter the search term in the search box and press Enter
search_box = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'theme-IYtZzqYPto8PhOx3ku3c')))
search_box.send_keys(search_term + Keys.RETURN)



# Load all products
number_of_products = int(wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'searchResultSummaryBar-AVnHBWRNB0_veFy34hco')))[1].text)

number_of_loaded_products = 0
while number_of_loaded_products < number_of_products:
    loaded_products = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'li[class*=productListContent][id]')))
    number_of_loaded_products = len(loaded_products)
    driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', loaded_products[-1])

# Get the link, name, price and seller of all the products
product = {key:[] for key in ['name','price','seller','url']}
product['name']  = [h3.text for h3 in driver.find_elements(By.CSS_SELECTOR, 'h3[data-test-id=product-card-name]')]
product['url']   = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[class*=ProductCard]')]
product['price'] = [float(div.text.replace('TL','').replace(',','.')) for div in driver.find_elements(By.CSS_SELECTOR, 'div[data-test-id=price-current-price]')]
for i,url in enumerate(product['url']):
    print(f'Search seller names {i+1}/{number_of_loaded_products}', end='\r')
    driver.get(url)
    product['seller'] += [wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.seller a'))).text]
    product['url'][i] = driver.current_url # useful to replace some long urls
    

# Sort by price in ascending order
product = pd.DataFrame(product, columns=['name', 'price', 'seller', 'url'])
product = pd.DataFrame(product).sort_values(by='price')
product = product.reset_index(drop=True)

# Save the product data to a CSV file
filename = f"{search_term.replace(' ', '_')}_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"


# Save the product data to a CSV file
product.to_csv(filename, index=False, encoding='utf-8-sig')


# Get the path to the directory containing the CSV files
dir_path = '.'

# Get a list of all CSV files in the directory
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv') and '_price_data' not in f]

# Initialize variables to hold information about the latest file
latest_file = None
latest_file_datetime = datetime.datetime.min

# Loop over each file in the directory
for file in csv_files:
    # Split the file name into parts using the "_" character as a delimiter
    file_parts = file.split("_")
    
    # Check if the file name contains a date and time string
    if len(file_parts) >= 3:
        # Extract the date and time from the file name
        file_date_str = file_parts[-2]
        file_time_str = file_parts[-1].replace(".csv", "")
        
        # Combine the date and time strings to create a datetime object
        file_datetime = datetime.datetime.strptime(file_date_str + "_" + file_time_str, "%Y-%m-%d_%H-%M-%S")
        
        # Check if this file is newer than the latest file we've found so far
        if file_datetime > latest_file_datetime:
            latest_file_datetime = file_datetime
            latest_file = file
    
# Delete all files in the directory except for the latest file
for file in csv_files:
    if file != latest_file:
        os.remove(os.path.join(dir_path, file))
        print(f"Removed old file: {file}")


# Create a function to get the current time in the required format
def get_time():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

# Set the tracking period to 3 times a day (morning, noon, and evening)
#---------------------------------------------------------------------------------------------------------------------------------------------------
tracking_period = ['12:52']

completed_days = 0
tracking_count = 0
day_price_data = []
price_data = []

def get_price():
    try:
        price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'offering-price')))
        price = float(price_element.find_element_by_xpath('./span[1]').text + '.' + price_element.find_element_by_xpath('./span[2]').text)
        return price
    except:
        return None

def get_time():
    return datetime.datetime.now()
    
    
def remaining_days_str():
    remaining_days = tracking_days - completed_days - 1
    return str(remaining_days)


def job():
    global completed_days, tracking_count
    
    current_time = datetime.datetime.now()
    print(f"Time just before tracking_period: {current_time}")
    if current_time.strftime('%H:%M') in tracking_period:
        driver.get( product['url'][0])
        current_price = get_price()

        if current_price is not None:
            day_price_data.append([get_time(), current_price])
            print(f"Price added at {get_time()}: {current_price}")
    
    tracking_count += 1
    print(f"Remaining days: {remaining_days_str()}")

    if tracking_count == len(tracking_period):
        completed_days += 1
        tracking_count = 0
        print(f"Day {completed_days} completed.")

    if completed_days == tracking_days:
        print("Product tracking finished.")
        return schedule.CancelJob



for time_str in tracking_period:
    if ':' in time_str:
        hour, minute = time_str.split(':')
    else:
        hour, minute = time_str, '00'
    print("time_str:", time_str)
    schedule.every().day.at(f"{hour}:{minute}").do(job)

print("Schedule started...")
while completed_days < tracking_days:
    schedule.run_pending()
    #print("Waiting for the next job...")
    time.sleep(50)  

day_price_df = pd.DataFrame(day_price_data, columns=['date', 'price'])
day_price_df['date'] = pd.to_datetime(day_price_df['date'])
price_data.append(day_price_df)
#---------------------------------------------------------------------------------------------------------------------------------------------------


# Concatenate all the day_price_df DataFrames and create a single DataFrame
price_df = pd.concat(price_data, ignore_index=True)

price_df['date'] = pd.to_datetime(price_df['date'])
price_df['time'] = price_df['date'].dt.time
price_df['date'] = price_df['date'].dt.date

final_price_df = pd.DataFrame(columns=['date', 'time', 'price'])

for date in price_df['date'].unique():
    
    # Get the dataframe for the current date
    daily_df = price_df[price_df['date'] == date]
    
    # If there is only one data point for the current date, append it to the final_price_df dataframe
    if len(daily_df) == 1:
        final_price_df = final_price_df.append({
            'date': daily_df['date'].iloc[0],
            'time': daily_df['time'].iloc[0],
            'price': daily_df['price'].iloc[0]
        }, ignore_index=True)
    
    # If all the prices for the current date are the same, get the latest time and append that data point to the final_price_df dataframe
    elif len(daily_df['price'].unique()) == 1:
        max_time = daily_df['time'].max()
        final_price_df = final_price_df.append({
            'date': date,
            'time': max_time,
            'price': daily_df[daily_df['time'] == max_time]['price'].iloc[0]
        }, ignore_index=True)
    
    # If there are multiple prices for the current date, get the latest time and append that data point to the final_price_df dataframe
    else:
        max_time = daily_df['time'].max()
        final_price_df = final_price_df.append({
            'date': date,
            'time': max_time,
            'price': daily_df[daily_df['time'] == max_time]['price'].iloc[0]
        }, ignore_index=True)
      

# Save the price data to a CSV file
filename = search_term.replace(' ', '_') + '_price_data.csv'
final_price_df.to_csv(filename, index=False, encoding='utf-8-sig')


# Remove old _price_data.csv files from directory
# Get the path to the directory containing the CSV files
dir_path = '.'

# Get a list of all CSV files in the directory
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv') and '_price_data' in f]

# Get the current date
today = datetime.date.today()

# Loop over each file in the directory
for file in csv_files:
    # Get the creation time of the file
    creation_time = os.path.getctime(os.path.join(dir_path, file))
    
    # Convert the creation time to a datetime object
    file_datetime = datetime.datetime.fromtimestamp(creation_time)
        
    # Check if this file is older than today
    if file_datetime.date() < today:
        os.remove(os.path.join(dir_path, file))
        print(f"Removed old file: {file}")
            
    # Check if this file is the latest file for today
    elif file_datetime.date() == today:
        # Check if there are multiple files for today
        if len(csv_files) > 1:
            # Remove all but the most recent file
            latest_file = max(csv_files, key=os.path.getctime)
            csv_files.remove(latest_file)
            for old_file in csv_files:
                os.remove(os.path.join(dir_path, old_file))
                print(f"Removed old file: {old_file}")
        else:
            print(f"No old files to remove for {file}")
    # If the file is from the future, just skip it
    else:
        pass

final_date = final_price_df['date'].max()  # latest day in data
start_date = final_price_df['date'].min()  # First day in data
elapsed_days = (final_date - start_date).days+1


#------------------------------------------------------------------------------------------------------------------------------
# Define the filenames using the search_term variable and the current date and time
now = datetime.datetime.now()

price_trend_filename = 'graphical_images_for_pdf/{} Price Trend {}.png'.format(search_term, now.strftime("%Y-%m-%d_%H-%M-%S"))
lowest_highest_prices_filename = 'graphical_images_for_pdf/{} Lowest Highest Prices {}.png'.format(search_term, now.strftime("%Y-%m-%d_%H-%M-%S"))
highlighted_prices_filename = 'graphical_images_for_pdf/{} Highlighted Prices {}.png'.format(search_term, now.strftime("%Y-%m-%d_%H-%M-%S"))
#------------------------------------------------------------------------------------------------------------------------------

condition=[]
unique_prices = final_price_df['price'].unique()
# Create a line plot of the price data
if not final_price_df.empty and final_price_df["price"].dtype != 'O':
   
    # Check if there is more than one unique price
    if len(unique_prices) > 1:
        condition1 = "condition 1: Price changes continuously"
        print("The condition in which the price is in a state of change.")
        condition.append(condition1)
        plt.plot(final_price_df['date'], final_price_df['price'])
        plt.title(f"\n\nPrice Trend of {search_term}\n", fontsize=16, fontweight='bold')
        plt.xlabel("\nDate\n", fontsize=14)
        plt.ylabel("Price (TL)\n", fontsize=14)
        plt.gca().xaxis.set_major_locator(DayLocator())
        plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
        plt.xticks(rotation=45)
        plt.gca().spines['top'].set_linewidth(2)
        plt.gca().spines['right'].set_linewidth(2)
        plt.gca().spines['bottom'].set_linewidth(2)
        plt.gca().spines['left'].set_linewidth(2)
        plt.savefig(price_trend_filename, dpi=500,bbox_inches='tight')  ###############
        plt.show()
    else:
        print(f"The price didn't change over the {elapsed_days} days: {unique_prices[0]} TL") ###

    # Create a bar plot of the lowest and highest prices
    if len(final_price_df) >= 1:
        fig, ax = plt.subplots()
        ax.bar(['Lowest Price', 'Highest Price'], [final_price_df['price'].min(), final_price_df['price'].max()])
        ax.set_title(f"\n\nLowest and Highest Prices of {search_term}\n", fontsize=16, fontweight='bold')
        ax.set_ylabel("Price (TL)\n", fontsize=14)
        ax.spines['top'].set_linewidth(2)
        ax.spines['right'].set_linewidth(2)
        ax.spines['bottom'].set_linewidth(2)
        ax.spines['left'].set_linewidth(2)
        plt.savefig(lowest_highest_prices_filename, dpi=500,bbox_inches='tight') # Grafiği PNG formatında kaydediyoruz. ##########
        plt.show()
        print(f"lowest highest prices image file was created as {lowest_highest_prices_filename}")
        # Create a line plot of the price data with the lowest and highest prices highlighted
    
        if len(final_price_df) >= 2 and final_price_df['price'].max() != final_price_df['price'].min():
            # Plot the price data
            plt.plot(final_price_df['date'], final_price_df['price'])

            # Add a title and axis labels
            plt.title(f"\n\nHighlighted Representation of the Highest and Lowest Prices of {search_term}\n", fontsize=16, fontweight='bold')
            plt.xlabel("\nDate\n", fontsize=14)
            plt.ylabel("Price (TL)\n", fontsize=14)

            plt.gca().xaxis.set_major_locator(DayLocator())
            plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
            plt.xticks(rotation=45)

            # Add a horizontal line for the lowest price
            plt.axhline(y=final_price_df['price'].min(), color='r', linestyle='-')

            # Add a horizontal line for the highest price
            plt.axhline(y=final_price_df['price'].max(), color='g', linestyle='-')

            # Set the line widths for the plot
            plt.gca().spines['top'].set_linewidth(2)
            plt.gca().spines['right'].set_linewidth(2)
            plt.gca().spines['bottom'].set_linewidth(2)
            plt.gca().spines['left'].set_linewidth(2)

            # Find the intersection points between the price data and the horizontal lines
            idx_min = np.argmin(final_price_df['price'])
            idx_max = np.argmax(final_price_df['price'])

            # Add a point for the lowest price intersection
            plt.plot(final_price_df['date'][idx_min], final_price_df['price'][idx_min], 'ro')

            # Add a point for the highest price intersection
            plt.plot(final_price_df['date'][idx_max], final_price_df['price'][idx_max], 'go')
            plt.savefig(highlighted_prices_filename, dpi=500, bbox_inches='tight') #############
            plt.show()


        else:
            condition2= "condition 2: One or constant price data point"
            print("There is only one or constant price data point. Cannot create the line plot with lowest and highest prices highlighted.")
            condition.append(condition2)
    else:
        condition3= "condition 3: Missing information to draw line and bar plot"
        print("There is only one price data point. Cannot create the line plot and the bar plot.")
        condition.append(condition3)
else:
    condition4= "condition 4: No price data information"
    print("There is no price data to create a plot.")
    condition.append(condition4)

# Remove old files from graphical_images_for_pdf -----------------------------------------------------------------------------------------------------------------------

folder_path = 'graphical_images_for_pdf'
# Split files with keywords
keyword_files = defaultdict(list)
for filename in os.listdir(folder_path):
    if 'Highlighted Prices' in filename:
        keyword_files['Highlighted Prices'].append(filename)
    elif 'Price Trend' in filename:
        keyword_files['Price Trend'].append(filename)
    elif 'Lowest Highest Prices' in filename:
        keyword_files['Lowest Highest Prices'].append(filename)

# Take action for each keyword
for keyword, files in keyword_files.items():
    
    if len(files) <= 1 and final_price_df['price'].nunique()!=1 :
        # keep if there is only 1 file
        continue
      
    elif len(files) == 1 and final_price_df['price'].nunique()== 1:
        file_path = os.path.join(folder_path, files[0])
        file_time = os.path.getmtime(file_path)
        current_time = time.time()
        if file_time < current_time-100: # possible delay 100 ms
            os.remove(file_path)
            continue

        
    else:
        # Sorting by date and time information
        sorted_files = sorted(files, key=lambda x: datetime.datetime.strptime(x.split('_')[-2].split(' ')[-1] + '_' + x.split('_')[-1].split('.')[0], "%Y-%m-%d_%H-%M-%S"), reverse=True)
        print("sorted_files:",sorted_files)
        # Keep the most recent file
        keep_file = sorted_files[0]
        keep_file_path = os.path.join(folder_path, keep_file)
        # Delete remaining files
        for file in sorted_files[1:]:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
        print(f"keeped file:{keyword}: {keep_file}")
        print(f"deleted file(s):{keyword}: {[os.path.join(folder_path, file) for file in sorted_files[1:]]}")

        
        
# Pdf section------------------------------------------------------------------------------------------------------------------

if "roboto-bold" or "roboto-regular" not in [fam.lower() for fam in pdfmetrics.getRegisteredFontNames()]:
    pdfmetrics.registerFont(TTFont("Roboto-Bold", "fonts\Roboto\Roboto-Bold.ttf"))
    pdfmetrics.registerFont(TTFont("Roboto-Regular", "fonts\Roboto\Roboto-Regular.ttf"))
else:
    print("Roboto-Bold/Roboto-Regular fonts have already been registered.")


styles = getSampleStyleSheet()

# Alt Bilgi Stili
footer_style = ParagraphStyle(
    name='footer',
    parent=styles['Normal'],
    fontName='Helvetica',
    fontSize=8, 
    textColor=colors.lightgrey,
    alignment=0,  # align to the left
)

# Footer text
footer_text = 'Created by Hepsiburada Price Tracker'
footer = Paragraph(footer_text, footer_style)

# Header style
header_style = ParagraphStyle(
    name='header',
    parent=styles['Normal'],
    fontName='Helvetica-Oblique',
    fontSize=10,
    textColor=colors.black,
    alignment=1
)

# Cover style
cover_style = ParagraphStyle(
    name='cover',
    parent=styles['Normal'],
    fontName='Roboto-Bold',
    textColor=colors.black,
    alignment=1
)


# Side header style

side_header_style = ParagraphStyle(
    name='side_header',
    parent=styles['Normal'],
    fontName='Roboto-Bold',
    fontSize=18,
    textColor=colors.black,
    leftIndent=-63, 
    spaceAfter=40 
)


# Result style
result_style = ParagraphStyle(
    name='result',
    parent=styles['Normal'],
    fontName='Roboto-Regular',
    fontSize=12,
    textColor=colors.black,
    leftIndent=-63,
    spaceAfter=20,
    bulletFontName='Helvetica',
    bulletFontSize=12,
    bulletIndent=-62,
    bulletText="\u25C6", # Add bullet
    leading=20 # Add line spacing
)


def generate_pdf(pdf_file, content, footer_style, header_style, cover_style, result_style, side_header_style):
   

    doc = SimpleDocTemplate(pdf_file, pagesize=(215.9 * mm, 279.4 * mm), topMargin=17)


    # cover text
    cover_text = f'<font size="30" color="black">{search_term} Analysis Results </font><br/><br/><br/><br/><br/><br/>'

    cover = Paragraph(cover_text, cover_style)

    # Add image for cover page
    image = Image('cover_template_for_pdf/cover_template.jpg', 9 * inch, 8.59 * inch)

    # Page content
    content = [cover, image, PageBreak()] + content 

    def page_number(canvas, doc):
        page_num = canvas.getPageNumber()
        canvas.setFillColorRGB(0.2, 0.2, 0.2)
        text = "{}".format(page_num)
        canvas.setFont("Helvetica", 10)  # Sayfa numarası yazı boyutu
        canvas.drawCentredString(doc.width / 2 + doc.leftMargin, 0.5 * inch, text)
        
        
        canvas.setFont("Helvetica", 8)  # Alt bilgi yazı boyutu
        canvas.setFillColorRGB(191/255, 87/255, 0)  # Turuncu rengi ayarla
        footer_width = canvas.stringWidth(footer_text, "Helvetica", 8)
        x = doc.leftMargin - 20 * mm  # Sola yasla ve sol taraftan 20 mm boşluk bırak
        canvas.drawString(x, 0.3 * inch, footer_text)  # Alt bilgiyi yazdır

        
    def onFirstPage(canvas, doc):
        canvas.saveState()
        page_number(canvas, doc)
        # Gray line
        canvas.setStrokeColor(colors.orange)
        canvas.setLineWidth(1)
        canvas.line(0, 0.75 * inch, doc.pagesize[0], 0.75 * inch)
        canvas.restoreState()

    def onLaterPages(canvas, doc):
        canvas.saveState()
        page_number(canvas, doc)
        # Gray line
        canvas.setStrokeColor(colors.orange)
        canvas.setLineWidth(1)
        canvas.line(0, 0.75 * inch, doc.pagesize[0], 0.75 * inch)
        canvas.restoreState()

    
     
        
    doc.build(content, onFirstPage=onFirstPage, onLaterPages=onLaterPages)
    return pdf_file


subheading1="1. Summary information"
Searched_product=f"Searched product: {search_term}"
Founded_product=f"The name of the lowest-priced product found for {search_term} search: {product['name'][0]}" 
Seller_offering_the_lowest_price_for_the_searched_product=f"Seller offering the lowest price for the searched product: {product['seller'][0]}"
Link_of_the_searched_product=f"Link of the searched product: {product['url'][0]}"
Duration_of_tracking_the_product=f"Duration of tracking the product:{tracking_days}"


subheading2="2. Graphic(s)"    
 
if  condition[0]=="condition 1: Price changes continuously":   
    spacer = Spacer(1, 60)
    # Add the content to the existing PDF
    content_list  = [Paragraph(subheading1, side_header_style), 
    Paragraph(Searched_product,result_style),
    Paragraph(Founded_product,result_style),
    Paragraph(Seller_offering_the_lowest_price_for_the_searched_product,result_style),
    Paragraph(Link_of_the_searched_product,result_style),
    Paragraph(Duration_of_tracking_the_product,result_style),
    Paragraph(subheading2, side_header_style),
           
    Image(price_trend_filename, width=275, height=275),
    Image(lowest_highest_prices_filename, width=275, height=275),   
    spacer,                
    Image(highlighted_prices_filename, width=370, height=330)]
   

    pdf_filename = "created_pdf_file/Analysis file for {search_term} {date}.pdf".format(search_term=search_term, date=now.strftime("%Y-%m-%d_%H-%M-%S"))

    generate_pdf(pdf_filename, content_list, footer_style, header_style, cover_style, result_style, side_header_style)    
                 
    
elif condition[0] == "condition 2: One or constant price data point" and final_price_df["price"].dtype == 'O':
   print("Since there is no price data for the product, the PDF file could not be created.")

elif condition[0] == "condition 2: One or constant price data point" and final_price_df["price"].dtype != 'O': 
    # Add the content to the existing PDF
    content_list  = [Paragraph(subheading1, side_header_style), 
    Paragraph(Searched_product,result_style),
    Paragraph(Founded_product,result_style),                
    Paragraph(Seller_offering_the_lowest_price_for_the_searched_product,result_style),
    Paragraph(Link_of_the_searched_product,result_style),
    Paragraph(Duration_of_tracking_the_product,result_style),
    Paragraph(subheading2, side_header_style),
           
    Image(lowest_highest_prices_filename, width=350, height=300)]
   

    pdf_filename = "created_pdf_file/Analysis file for {search_term} {date}.pdf".format(search_term=search_term, date=now.strftime("%Y-%m-%d_%H-%M-%S"))

    generate_pdf(pdf_filename, content_list, footer_style, header_style, cover_style, result_style, side_header_style)
    
    
    
elif condition[0] == "condition 3: Missing information to draw line and bar plot":
   print("There is only one price data point. Cannot create the line plot and the bar plot, the PDF file could not be created.")     
   
elif condition[0] == "condition 4: No price data information":
   print("There is no price data to create a plot, the PDF file could not be created.")       
    

# Send e-mail--------------------------------------------------------------------------------------------------------------------    
    
subject = f"Price Tracker Analysis Results for {search_term}"
body = "The results are in the attachment."
pdf_name= "created_pdf_file/Analysis file for Pınar Süt 1lt 2023-04-18_10-55-04.pdf"

# Pass the file name as a string in a list
files = [pdf_filename]

try:
    # Service account JSON key file
    service_account_file = 'files_for_email_sending/service_account_gmail_auth.json'


    with open('files_for_email_sending/email_config.json', 'r') as f:
        email_config = json.load(f)

    user_email = email_config['user_email']
    to_email = email_config['to_email']

except FileNotFoundError:
    print("Error: file(s) for email sending could not be found.")
    

def send_email(to: str, subject: str, body: str, files: List[str] = None):
 

    # Create credentials object
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file,
        scopes=['https://www.googleapis.com/auth/gmail.send',
                'https://www.googleapis.com/auth/gmail.compose',
                'https://www.googleapis.com/auth/gmail.readonly'])

    # Impersonate the user
    delegated_credentials = credentials.with_subject(user_email)

    # Create the Gmail API client
    gmail = build('gmail', 'v1', credentials=delegated_credentials)

    # Initialize the message object
    message = MIMEMultipart()
    message['to'] = to
    message['subject'] = subject

    # Add body to the message
    message.attach(MIMEText(body))

    # Add files to the message
    if files:
        for file_path in files:
            with open(file_path, 'rb') as f:
                file_content = f.read()
            file_name = os.path.basename(file_path)
            file = MIMEApplication(file_content, _subtype='pdf')
            file.add_header('content-disposition', 'attachment', filename=file_name)
            message.attach(file)

    # Send the message
    try:
        create_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
        send_message = gmail.users().messages().send(userId='me', body=create_message).execute()
        print(F'Email was sent to "{to}" with Email Id: {send_message["id"]}')
    except HttpError as error:
        print(F'An error occurred: {error}')
        send_message = None

    return send_message

try:
    send_email(to_email, subject, body,files)
except FileNotFoundError:
    print(f"Error: The {pdf_name} that was supposed to be sent via email could not be found.")

# Close the website tab to avoid errors
driver.quit()   
